In [228]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [210]:
years = np.arange(2018, 2024)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
path_base = Path('data')
directories = [path_base / str(year) / month for year in years for month in months]

dfs = []
for dir in directories:
    if dir.exists():
        for file in dir.iterdir():
            if file.is_file() and file.name.startswith('y'):
                df = wrangle(file)
                dfs.append(df)




In [223]:
biggie

,Parameter,Site Name,0,1,2,3,4,5,6,7,...,18,19,20,21,22,23,Avg,Max,Hr. of Max,Date
3,01 OZONE PPM,Alpine,.049,C,C,.047,.048,.045,.043,.043,...,.044,.042,.042,.044,.042,.046,0.045,.056,14,1/1/2018
4,01 OZONE PPM,Chula Vista,.002,C,C,.002,.004,.004,.002,.003,...,.021,.011,.003,.004,.012,.016,0.019,.048,13,1/1/2018
5,01 OZONE PPM,Del_Mar,,,,,,,,,...,,,,,,,NaN,,,1/1/2018
6,01 OZONE PPM,Downtown,,,,,,,,,...,,,,,,,NaN,,,1/1/2018
7,01 OZONE PPM,El Cajon Les,.004,C,C,C,.003,.003,.003,.004,...,.019,.009,.006,.004,.003,.003,0.020,.052,14,1/1/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,PM10 STD,Kearny Mesa,19.8,17.3,17.1,15.0,15.1,11.1,7.9,8.3,...,20.9,19.7,14.8,11.6,11.9,23.6,13.000,23.6,23,Parameter
103,PM10 STD,Otay Mesa Dvn,21.8,33.7,45.8,31.1,26.9,16.9,17.5,27.6,...,22.2,25.6,20.6,22.8,62.4,31.2,23.700,62.4,22,Parameter
104,PM10 STD,Pendleton,16.3,16.7,15.3,16.1,14.5,13.5,12.3,12.7,...,37.7,21.9,12.3,12.4,14.5,15.5,17.000,37.7,18,Parameter
105,PM10 STD,San Ysidro,50.7,55.8,41.8,40.3,35.9,35.2,35.8,24.7,...,34.3,31.7,26.6,26.3,22.1,55.4,30.400,55.8,01,Parameter


In [126]:
black_carbon = pd.read_csv('data/Hourly-Records-2022-12-05-to-2024-10-16.csv', dtype={'Units': str})
black_carbon['County'].value_counts()

C:\Users\banga\AppData\Local\Temp\ipykernel_82032\418190281.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  black_carbon = pd.read_csv('data/Hourly-Records-2022-12-05-to-2024-10-16.csv', dtype={'Units': str})


County
San Diego        91928
Contra Costa        19
Sacramento          18
Imperial            14
Los Angeles          5
Fresno               4
San Francisco        3
Kern                 2
Riverside            2
Inyo                 1
Name: count, dtype: int64

In [130]:
bc_filt = black_carbon[black_carbon['Units'] == 'ug/m3 (LC)']
bc_filt = bc_filt[bc_filt['County'] == 'San Diego']
bc_filt.groupby('Site')['MeasuredValue'].max()


Site
10th Ave Marine Terminal       32.70083
Caltrans at Boston Ave         12.57166
Chicano Park                   10.88833
Downtown Sherman Elementary     9.42500
Oceanview Blvd                 13.36916
Otay Mesa - Donovan            18.72000
San Ysidro                     30.55166
Name: MeasuredValue, dtype: float64

In [230]:
biggie = pd.concat(dfs)

biggie['Avg'] = biggie['Avg'].replace({'':np.nan, 'M':np.nan, '<':np.nan, 'P':np.nan})
biggie['Avg'] = biggie['Avg'].astype(float)
biggie['Site Name'] = biggie['Site Name'].str.title()
grouped  = biggie.groupby(['Site Name', 'Parameter'])['Avg'].mean()
each_row = grouped.reset_index()
each_row_json = each_row.to_json(orient='records')

# Write JSON to a file
with open("everything.json", "w") as outfile:
    json.dump(each_row_json, outfile)

In [231]:
each_row

,Site Name,Parameter,Avg
0,10Th Ave Marine Term,BCarbon ug/m3,0.795430
1,Alpine,01 OZONE PPM,0.041960
2,Alpine,02 NOX PPM,0.004621
3,Alpine,03 NO2 PPB,3.390787
4,Alpine,03 NO2 PPM,0.004066
...,...,...,...
164,San Ysidro,10 INTMP °C,22.945652
165,San Ysidro,11 PM2.5 µg/M3,16.758696
166,San Ysidro,16 AMBTEMP °F,63.149451
167,San Ysidro,BCarbon ug/m3,1.214692


In [209]:
import pandas as pd

# Read the file with custom parsing
file_path = '8_yesterday_20240917.csv'
def wrangle(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            lines = file.readlines()
    
    # Extract date and irrelevant lines
    date_info = lines.pop(0).strip()
    lines.pop(0)
    
    # Initialize data structure
    data = []

    # Parse the remaining lines
    for line in lines:
        split_line = line.strip().split(',')
        if len(split_line) >= 29:  # Ensure there are at least 29 fields
            data.append(split_line[:29])  # Adjust if different number of fields per row

    # Create DataFrame from the parsed data
    columns = [
        'Parameter', 'Site Name', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
        '17', '18', '19', '20', '21', '22', '23', 'Avg', 'Max', 'Hr. of Max'
    ]
    df = pd.DataFrame(data, columns=columns)
    df['Date'] =  df.iloc[0]['Parameter']
    df = df.iloc[3:]
    df['Parameter'] = df['Parameter'].replace('', np.nan)
    df['Avg'] = df['Avg'].replace({'':np.nan,'<':np.nan, 'M':np.nan, 'P':np.nan}).astype(float)

    # Forward fill the 'Parameter' column
    df['Parameter'] = df['Parameter'].ffill()
    return df
    


In [153]:
geo = pd.read_csv("2015_San_Diego_County_Demographics_-_County_Population_by_Age_and_Gender.csv")
geo[['Geography', 'Percent 65+']]


,Geography,Percent 65+
0,Central Region,9.6
1,Central San Diego,10.3
2,Mid-City,8.1
3,Southeastern San Diego,10.2
4,East Region,13.3
5,Alpine,18.5
6,El Cajon,12.3
7,Harbison Crest,18.6
8,Jamul,13.1
9,La Mesa,15.0


In [ ]:
df.merge(geo, left_on='')